# Point Cloud Neural Operator
![structure of PCNO](structure.svg)

The Point Cloud Neural Operator (PCNO) is a neural network - based surrogate model framework meticulously designed to approximate solution maps of parametric partial differential equations (PDEs) on intricate and variable geometries. Given an abstract parametric PDE $$\mathcal{R}(u, a) = 0$$ defined on $\Omega \subset \mathbb{R}^{d}$, where $\mathcal{R}$ represents a generalized differential operator, $a: \mathbb{R}^{d} \to \mathbb{R}^{d_{a}}$ stands for the parameter function, and $u(x): \mathbb{R}^{d} \to \mathbb{R}^{d_{u}}$ denotes the solution function. The overarching objective of PCNO is to approximate the solution map 
$$\mathcal{G}^{\dagger}:(a, \Omega) \mapsto u.$$

## Point Cloud Neural Operator

The Point Cloud neural layer $\mathcal{L}$ serves to map the input function $f_{\rm in}: \Omega \rightarrow \mathbb{R}^{d_{\rm in}}$ to the output function $f_{\rm out}: \Omega \rightarrow \mathbb{R}^{d_{\rm out}}$. This mapping is defined as follows:
$$
\begin{equation*}
\begin{split}
    &\mathcal{L} : f_{\rm in} \mapsto f_{\rm out}, \\
    &f_{\rm out}(x) =  \sigma \Bigl(W^{l} f_{\rm in}(x) + b + \sum_k\int_{\Omega} e^{-2\pi i\frac{k}{L}(x - y)}W_k^vf_{\rm in}(y)\rho(y,\Omega) {\rm d}y + W^g\tilde{\nabla}f_{in}\Bigr).\
\end{split}
\end{equation*}
$$
Here, $W^{l}\in\mathbb{R}^{d_{\rm out}\times d_{\rm in}}$, $b\in\mathbb{R}^{d_{\rm out}}$, $W_k^v\in\mathbb{C}^{d_{\rm out}\times d_{\rm in}},W_g \in\mathbb{R}^{d_{\rm out}\times (d\cdot d_{\rm in})}$ all operate pointwise.
- The linear term $W^{l} f_{\rm in}(x) + b$ represent the standard linear transformation.
- The integral term $\sum_k\int_{\Omega} e^{-2\pi i\frac{k}{L}(x - y)}W_k^vf_{\rm in}(y)\rho(y,\Omega) {\rm d}y$ captures the long - range dependencies in the domain $\Omega$. 
- The gradient term $W^g\tilde{\nabla}f_{in}$incorporates the local gradient information of the input function $f_{in}$, which can help the network better understand the local behavior of the data.



## Parametrization and Computation 

### integral operator

We establish a connection between the geometry $\Omega$ and a density function $\rho(x ; \Omega)$ defined on $\Omega$. The integral operator is formulated as:
$$
f_{\rm out}(x)=\sum_k \int_{\Omega} e^{2\pi i\frac{k}{L}(x - y)}W_k^vf_{\rm in}(y)\rho(y;\Omega) {\rm d}y
$$
where $W_k^{v} \in \mathbb{C}^{d_{out} \times d_{in}}$ is a parameter matrix. The density function $\rho(x ; \Omega)$ plays a crucial role in ensuring the stability of the integral during the training process. It also simplifies the numerical integration, as it allows for a more efficient approximation of the integral over the complex domain $\Omega$.

Consider a set of bases $\{\phi_k\}$ on $\Omega$. We define the weighted bases $\{\phi_{w,k}\}$ as
$$\phi_k(x) = e^{2\pi i\frac{k}{L}x}, \quad \phi_{w,k}(x_i) =  \overline{\phi_k(x_i)}\rho(x_i;\Omega){\rm d}\Omega_i$$
Then, we can rewrite the integral operator as follows:
$$
\begin{align*}
f_{\rm out}(x)&=\sum_k \int_{\Omega} e^{2\pi i\frac{k}{L}(x - y)}W_k^vf_{\rm in}(y)\rho(y;\Omega) {\rm d}y\\
&\approx  \sum_k\sum_i \phi_k(x)\overline{\phi_k(y_i)} W_k^vf_{\rm in}(y_i)\rho(y_i;\Omega) {\rm d}\Omega_i\\
&=  \sum_k \phi_k(x)W_k^v\sum_i\phi_{w,k}(y_i) f_{\rm in}(y_i)\\
&= \sum_k \phi_k(x)W_k^v\hat{f_{\rm in}}[k]\\
\end{align*}
$$
Since we are only concerned with the real part, to avoid complex - valued arithmetic operations, we can separately utilize the sine and cosine components. The computational process can be divided into three steps:

- **Step 1: Fourier - like Coefficient Computation**

Compute 
$$
\begin{align*}  
\hat{f}_{\rm in}^c[k] &= \langle\phi_{w,k}^c,f_{\rm in} \rangle ,\\
\hat{f}_{\rm in}^s[k] &= \langle\phi_{w,k}^s,f_{\rm in} \rangle,\\
\hat{f}_{\rm in}[0] &= \langle\phi_{w,0},f_{\rm in} \rangle.
\end{align*}$$
Here, $\phi_{w,k}^c$ and $\phi_{w,k}^s$ are the real part and the imaginary part of $\phi_{w,k}$ respectively. 

The corresponding codes are:
```
x_c_hat =  torch.einsum("bix,bxkw->bikw", x, wbases_c)
x_s_hat = -torch.einsum("bix,bxkw->bikw", x, wbases_s)
x_0_hat =  torch.einsum("bix,bxkw->bikw", x, wbases_0)
```

- **Step 2: Transformed Coefficient Computation**

Compute
$$
\begin{align*}  
 &\widehat{Rf}^c_{\rm in}[k]  = R^c(k)\hat{f}^c_{\rm in}[k] - R^s(k)\hat{f}^s_{\rm in}[k] \\ 
 &\widehat{Rf}^s_{\rm in}[k] =  R^c(k)\hat{f}^s_{\rm in}[k] + R^s(k)\hat{f}^c_{\rm in}[k]  \\
 &\widehat{Rf}_{\rm in}[0] =  R^c(0)\hat{f}_{\rm in}[0]
\end{align*}$$
where $R^c(k)$ and $R^s(k)$ are real - valued martixs. The multiplication methods here originate from the complex multiplication of $W_k^v$.

The corresponding codes are:
```
f_c_hat = torch.einsum("bikw,iokw->bokw", x_c_hat, weights_c) - torch.einsum("bikw,iokw->bokw", x_s_hat, weights_s)
f_s_hat = torch.einsum("bikw,iokw->bokw", x_s_hat, weights_c) + torch.einsum("bikw,iokw->bokw", x_c_hat, weights_s)
f_0_hat = torch.einsum("bikw,iokw->bokw", x_0_hat, weights_0) 
```
- **Step 3: Output Function Reconstruction**
$$
\begin{align*}
f_{\rm out} = \widehat{Rf}_{\rm in}[0]\phi_0 + 2\sum_k \widehat{Rf}_{\rm in}^c[k]\phi_k^c - 2\sum_k \widehat{Rf}_{\rm in}^s[k]\phi_k^s
\end{align*}$$
the factor of 2 and the negative sign here both come from taking the real part of the original complex multiplication.

The corresponding codes are:

```
x = torch.einsum("bokw,bxkw->box", f_0_hat, bases_0)  + 2*torch.einsum("bokw,bxkw->box", f_c_hat, bases_c) -  2*torch.einsum("bokw,bxkw->box", f_s_hat, bases_s)
```
